<a href="https://colab.research.google.com/github/minsaee/ai_chat_python/blob/master/406_kakao%EB%A9%94%EC%8B%9C%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#기본설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_python'

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_python


In [ ]:
import requests

url = 'https://kauth.kakao.com/oauth/token'
data = {
    'grant_type':'authorization_code',
    'client_id':'REST API Key',
    'redirect_uri' : 'https://localhost.com',
    'code':'code'
}


response = requests.post(url, data = data)

if response.status_code != 200:
  print('error! because', response.json())
else:
  tokens = response.json()
  print(tokens)

#토큰 관리하기

In [45]:
import json
import requests
import datetime
import os

# 카카오 토큰을 저장할 파일명
KAKAO_TOKEN_FILENAME = "./res/kakao_message/kakao_token.json"

# 저장하는 함수
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

# 읽어오는 함수
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)

    return tokens

# refresh_token으로 access_token 갱신하는 함수
def update_tokens(app_key, filename) :
    tokens = load_tokens(filename)

    url = "https://kauth.kakao.com/oauth/token"
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    response = requests.post(url, data=data)

    # 요청에 실패했다면,
    if response.status_code != 200:
        print("error! because ", response.json())
        tokens = None
    else: # 성공했다면,
        print(response.json())
        # 기존 파일 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename+"."+ now
        os.rename(filename, backup_filename)
        # 갱신된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)

    return tokens

# 토큰 저장
save_tokens(KAKAO_TOKEN_FILENAME, tokens)

# 토큰 업데이트 -> 토큰 저장 필수!
#KAKAO_APP_KEY = "<REST_API 앱키를 입력하세요>"
#tokens = update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)
#save_tokens(KAKAO_TOKEN_FILENAME, tokens)

# 구현하기

텍스트 메시지 템플릿을 사용하여 카카오톡 메시지 보내기.  
https://developers.kakao.com/docs/latest/ko/message/rest-api#default-template-msg

In [47]:
import json
import requests

# 저장된 토큰 정보를 읽어오기.
tokens = load_tokens(KAKAO_TOKEN_FILENAME)

# 텍스트 메시지 보내기 url 설정하기.
url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'

# request parameter 설정하기.
headers = {
    'Authorization': 'Bearer '+tokens['access_token']
}

data = {
    'template_object' : json.dumps({
        'object_type' : 'text',
        'text': 'Hello, World!',
        'link':{
            'web_url':'https://www.naver.com',
            'mobile_web_url':'https://m.naver.com'
        }
    })
}

# 나에게 카카오톡 메시지 보내기 요청 (type = text)
response = requests.post(url, headers= headers, data=data)
print(response.status_code)

if response.status_code != 200:
  print('error! because', response.json())
else:
  print('메시지를 성공적으로 보냈습니다.')

200
메시지를 성공적으로 보냈습니다.


In [53]:
import json
import requests

# 저장된 토큰 정보를 읽어오기.
tokens = load_tokens(KAKAO_TOKEN_FILENAME)

# 리스트 메시지 보내기 url 설정하기.
url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'

# request parameter 설정하기.
headers = {
    'Authorization': 'Bearer '+tokens['access_token']
}

template = {
    "object_type" : "list",
    "header_title" : "초밥 사진",
    "header_link" : {
        "web_url" : "https://www.naver.com",
        "mobile_web_url" : "https://www.naver.com"
    },
    "contents" : [
        {
            "title" : "1. 광어초밥",
            "description" : "광어는 맛있다",
            "image_url" : "https://search1.kakaocdn.net/argon/0x200_85_hr/8x5qcdbcQwi",
            "image_width" : 50, "image_height" : 50,
            "link" : {
                "web_url" : "https://www.naver.com",
                "mobile_web_url" : "https://www.naver.com"
            }
        },
        {
            "title" : "2. 참치초밥",
            "description" : "참치는 맛있다",
            "image_url" : "https://search2.kakaocdn.net/argon/0x200_85_hr/IjIToH1S7J1",
            "image_width" : 50, "image_height" : 50,
            "link" : {
                "web_url" : "https://www.naver.com",
                "mobile_web_url" : "https://www.naver.com"
            }
        }

    ],
    "buttons" : [
        {
            "title" : "웹으로 이동",
            "link" : {
                "web_url" : "https://www.naver.com",
                "mobile_web_url" : "https://www.naver.com"
            }
        }
    ]

}

data = {
    'template_object' : json.dumps(template)
}

# 나에게 카카오톡 메시지 보내기 요청 (type = list)
response = requests.post(url, headers= headers, data=data)
print(response.status_code)

if response.status_code != 200:
  print('error! because', response.json())
else:
  print('메시지를 성공적으로 보냈습니다.')

200
메시지를 성공적으로 보냈습니다.


In [54]:
%%writefile kakao_utils.py

import json
import os
import datetime
import requests

# 저장하는 함수
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

# 읽어오는 함수
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)

    return tokens

# refresh_token으로 access_token 갱신하는 함수
def update_tokens(app_key, filename) :
    tokens = load_tokens(filename)

    url = "https://kauth.kakao.com/oauth/token"
    data = {
        "grant_type" : "refresh_token",
        "client_id"  : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    response = requests.post(url, data=data)

    # 요청에 실패했다면,
    if response.status_code != 200:
        print("error! because ",  response.json())
        tokens = None
    else: # 성공했다면,
        print(response.json())
        # 기존 파일 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename+"."+ now
        os.rename(filename, backup_filename)
        # 갱신된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)

    return tokens

# 메시지 전송 함수
def send_message(filename, template):
    tokens = load_tokens(filename)

    headers = {
        "Authorization": "Bearer " + tokens['access_token']
    }

    # JSON 형식 -> 문자열 변환
    payload = {
        "template_object" : json.dumps(template)
    }

    # 카카오톡 보내기
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    res = requests.post(url, data=payload, headers=headers)

    return res

Writing kakao_utils.py


In [ ]:
import json
import requests
import kakao_utils

# 토큰 가져오기.
KAKAO_TOKEN_FILENAME = 'res/kakao_message/kakao_token.json'

# REST API KEY 가져오기.
KAKAO_APP_KEY = 'REST API Key'

# 토큰 업데이트하기.
tokens = kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

# 토큰 저장하기.
save_tokens(KAKAO_TOKEN_FILENAME, tokens)

# 텍스트 템플릿 만들기.
template = {
        'object_type' : 'text',
        'text': 'Hello, World!',
        'link':{
            'web_url':'https://www.naver.com',
            'mobile_web_url':'https://m.naver.com'
        }
}


# 나에게 카카오톡 메시지 보내기 요청 (text type)
response = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
print(response.status_code)

if response.status_code != 200:
  print('error! because', response.json())
else:
  print('메시지를 성공적으로 보냈습니다.')